<h1> AdaBoost Classifier using Decision tree as base and for PhishingVsBenignURL data set</h1>

In [1]:
import pandas as pd 
import numpy as np 
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Reading the csv and making a dataframe from the data
pvb = pd.read_csv(r"C:\Users\myste\Documents\MSCS\Fall 2020\ML\DataSetForPhishingVSBenignUrl.csv")

In [3]:
pvb.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,1,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement


<h2>Encoding the values of Phishing vs Non phishing to use as target variables</h2>

In [4]:
conditions = [(pvb['URL_Type_obf_Type'] == 'phishing'), (pvb['URL_Type_obf_Type'] != 'phishing')]
# create a list of the values we want to assign for each condition
values = [0,1]
# create a new column and use np.select to assign values to it using our lists as arguments
pvb['obf_Type'] = np.select(conditions, values)
# display updated DataFrame
pvb.head()

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,URL_Type_obf_Type,obf_Type
0,0,4,5,5.5,14,4.400000,4,8,3,0,...,0,-1,0.726298,0.784493,0.894886,0.850608,NaN,-1.0,Defacement,1
1,0,4,5,5.5,14,6.000000,4,12,4,0,...,0,-1,0.688635,0.784493,0.814725,0.859793,0.0,-1.0,Defacement,1
2,0,4,5,5.5,14,5.800000,4,12,5,0,...,0,-1,0.695049,0.784493,0.814725,0.801880,0.0,-1.0,Defacement,1
3,0,4,12,5.5,14,5.500000,4,32,16,0,...,0,-1,0.640130,0.784493,0.814725,0.663210,0.0,-1.0,Defacement,1
4,0,4,6,5.5,14,7.333334,4,18,11,0,...,0,-1,0.681307,0.784493,0.814725,0.804526,0.0,-1.0,Defacement,1


In [5]:
pvb=pvb.drop(columns='URL_Type_obf_Type')

<h2>Dropping the rows that have NaNs or Infinity as values</h2>

In [6]:
#Dropping the rows for a better and more accurate analysis
pvb.replace([np.inf, -np.inf], np.nan)
pvb.dropna(inplace=True)
pvb

,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath,obf_Type
35,0,4,4,6.250000,17,3.750000,4,4,2,0,...,1,0,-1,0.726945,0.768811,0.916667,1.000000,1.000000,-1.000000,1
37,22,4,10,6.250000,17,3.500000,4,19,10,0,...,9,8,7,0.686486,0.768811,0.916667,0.748105,0.757206,0.749167,1
38,23,4,10,6.250000,17,3.500000,4,19,10,0,...,9,8,7,0.687286,0.768811,0.916667,0.747622,0.756298,0.748268,1
39,22,4,10,6.250000,17,3.500000,4,19,10,0,...,9,8,7,0.683476,0.768811,0.916667,0.742090,0.750292,0.741506,1
40,23,4,10,6.250000,17,3.500000,4,19,10,0,...,9,8,7,0.687286,0.768811,0.916667,0.747622,0.756298,0.748268,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36702,29,4,14,5.750000,12,3.666667,4,20,24,3,...,3,2,7,0.690555,0.791265,0.777498,0.690227,0.656684,0.796205,1
36703,0,4,13,3.750000,8,8.461538,4,24,23,0,...,16,15,-1,0.665492,0.820010,0.879588,0.674400,0.674671,-1.000000,1
36704,58,3,27,6.666666,16,3.375000,3,41,34,20,...,8,7,9,0.656807,0.801139,0.684777,0.713622,0.717187,0.705245,1
36705,35,3,13,4.333334,9,3.600000,3,15,13,7,...,9,8,3,0.725963,0.897617,0.871049,0.745932,0.758824,0.790772,1


<h2>Splitting the data into target varible and features</h2>

In [7]:
#Putting encoded 'URL_Type_obf_Type' as the target output and all the other columns as features for training in X
y = pvb[['obf_Type']] 
X = pvb.drop(['obf_Type', ], axis = 1)
print(y.shape, X.shape)
X

(18982, 1) (18982, 79)


,Querylength,domain_token_count,path_token_count,avgdomaintokenlen,longdomaintokenlen,avgpathtokenlen,tld,charcompvowels,charcompace,ldl_url,...,SymbolCount_Directoryname,SymbolCount_FileName,SymbolCount_Extension,SymbolCount_Afterpath,Entropy_URL,Entropy_Domain,Entropy_DirectoryName,Entropy_Filename,Entropy_Extension,Entropy_Afterpath
35,0,4,4,6.250000,17,3.750000,4,4,2,0,...,1,1,0,-1,0.726945,0.768811,0.916667,1.000000,1.000000,-1.000000
37,22,4,10,6.250000,17,3.500000,4,19,10,0,...,1,9,8,7,0.686486,0.768811,0.916667,0.748105,0.757206,0.749167
38,23,4,10,6.250000,17,3.500000,4,19,10,0,...,1,9,8,7,0.687286,0.768811,0.916667,0.747622,0.756298,0.748268
39,22,4,10,6.250000,17,3.500000,4,19,10,0,...,1,9,8,7,0.683476,0.768811,0.916667,0.742090,0.750292,0.741506
40,23,4,10,6.250000,17,3.500000,4,19,10,0,...,1,9,8,7,0.687286,0.768811,0.916667,0.747622,0.756298,0.748268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36702,29,4,14,5.750000,12,3.666667,4,20,24,3,...,11,3,2,7,0.690555,0.791265,0.777498,0.690227,0.656684,0.796205
36703,0,4,13,3.750000,8,8.461538,4,24,23,0,...,2,16,15,-1,0.665492,0.820010,0.879588,0.674400,0.674671,-1.000000
36704,58,3,27,6.666666,16,3.375000,3,41,34,20,...,14,8,7,9,0.656807,0.801139,0.684777,0.713622,0.717187,0.705245
36705,35,3,13,4.333334,9,3.600000,3,15,13,7,...,1,9,8,3,0.725963,0.897617,0.871049,0.745932,0.758824,0.790772


<h2>Splitting into test and train data</h2>

In [8]:
#Splitting the data into testing and training (20: 80) using a randomly generated random state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = np.random)
y_test=y_test.astype('int')
y_train=y_train.astype('int')
print(X_train.shape, y_train.shape) 
print(X_test.shape, y_test.shape)

(15185, 79) (15185, 1)
(3797, 79) (3797, 1)


<h1>Gini Impurity</h1>

<h2>Max Depth = 1</h2>

In [9]:
gd1 = DecisionTreeClassifier(criterion = "gini",random_state = 50, max_depth=1, min_samples_leaf=5)
ada1=AdaBoostClassifier(gd1)
ada1.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ada1.predict(X_test), y_test))

Accuracy: 0.9194100605741374


<h2> Max Depth = 3 </h2>

In [10]:
gd3 = DecisionTreeClassifier(criterion = "gini",random_state = 50, max_depth=3, min_samples_leaf=5)
ada3=AdaBoostClassifier(gd3)
ada3.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ada3.predict(X_test), y_test))

Accuracy: 0.9639188833289439


<h2> Max Depth = 6 </h2>

In [11]:
gd6 = DecisionTreeClassifier(criterion = "gini",random_state = 50, max_depth=6, min_samples_leaf=5)
ada6=AdaBoostClassifier(gd6)
ada6.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ada6.predict(X_test), y_test))

Accuracy: 0.9654990782196471


<h2> Max Depth = 9 </h2>

In [12]:
gd9 = DecisionTreeClassifier(criterion = "gini",random_state = 50, max_depth=9, min_samples_leaf=5)
ada9=AdaBoostClassifier(gd9)
ada9.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ada9.predict(X_test), y_test))

Accuracy: 0.9739267843033974


<h2> Max Depth = 12 </h2>

In [13]:
gd12 = DecisionTreeClassifier(criterion = "gini",random_state = 50, max_depth=12, min_samples_leaf=5)
ada12=AdaBoostClassifier(gd12)
ada12.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ada12.predict(X_test), y_test))

Accuracy: 0.9757703450092178


<h2> Max Depth = 15 </h2>

In [14]:
gd15 = DecisionTreeClassifier(criterion = "gini",random_state = 50, max_depth=15, min_samples_leaf=5)
ada15=AdaBoostClassifier(gd15)
ada15.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ada15.predict(X_test), y_test))

Accuracy: 0.976033710824335


<h2> Max Depth = 18 </h2>

In [15]:
gd18 = DecisionTreeClassifier(criterion = "gini",random_state = 50, max_depth=18, min_samples_leaf=5)
ada18=AdaBoostClassifier(gd18)
ada18.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ada18.predict(X_test), y_test))

Accuracy: 0.973400052673163


<h1>Entropy Impurity</h1>

<h2>Max Depth = 1</h2>

In [16]:
ed1 = DecisionTreeClassifier(criterion = "entropy",random_state = 50, max_depth=1, min_samples_leaf=5)
ade1=AdaBoostClassifier(ed1)
ade1.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ade1.predict(X_test), y_test))

Accuracy: 0.9130892810113247


<h2>Max Depth = 3</h2>

In [17]:
ed3 = DecisionTreeClassifier(criterion = "entropy",random_state = 50, max_depth=3, min_samples_leaf=5)
ade3=AdaBoostClassifier(ed3)
ade3.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ade3.predict(X_test), y_test))

Accuracy: 0.9581248353963655


<h2>Max Depth = 6</h2>

In [18]:
ed6 = DecisionTreeClassifier(criterion = "entropy",random_state = 50, max_depth=6, min_samples_leaf=5)
ade6=AdaBoostClassifier(ed6)
ade6.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ade6.predict(X_test), y_test))

Accuracy: 0.9657624440347643


<h2>Max Depth = 9</h2>

In [19]:
ed9 = DecisionTreeClassifier(criterion = "entropy",random_state = 50, max_depth=9, min_samples_leaf=5)
ade9=AdaBoostClassifier(ed9)
ade9.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ade9.predict(X_test), y_test))

Accuracy: 0.9736634184882802


<h2>Max Depth = 12</h2>

In [20]:
ed12 = DecisionTreeClassifier(criterion = "entropy",random_state = 50, max_depth=12, min_samples_leaf=5)
ade12=AdaBoostClassifier(ed12)
ade12.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ade12.predict(X_test), y_test))

Accuracy: 0.976033710824335


<h2>Max Depth = 15</h2>

In [21]:
ed15 = DecisionTreeClassifier(criterion = "entropy",random_state = 50, max_depth=15, min_samples_leaf=5)
ade15=AdaBoostClassifier(ed15)
ade15.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ade15.predict(X_test), y_test))

Accuracy: 0.9762970766394522


<h2>Max Depth = 18</h2>

In [22]:
ed18 = DecisionTreeClassifier(criterion = "entropy",random_state = 50, max_depth=18, min_samples_leaf=5)
ade18=AdaBoostClassifier(ed18)
ade18.fit(X_train, y_train)
print("Accuracy:", metrics.accuracy_score(ade18.predict(X_test), y_test))

Accuracy: 0.9736634184882802


<h1>Previous Results for Decision Tree</h1> 

In [23]:
results = {'Depth': [1,2,3,4,5,6],
        'Accuracy_Gini': [0.785,0.843,0.874,0.891, 0.897,0.912],
        'Accuracy_Entropy': [0.785,0.843,0.874,0.874,0.895,0.906]
          }
df = pd.DataFrame(results, columns = ['Depth', 'Accuracy_Gini', 'Accuracy_Entropy'])
df

,Depth,Accuracy_Gini,Accuracy_Entropy
0,1,0.785,0.785
1,2,0.843,0.843
2,3,0.874,0.874
3,4,0.891,0.874
4,5,0.897,0.895
5,6,0.912,0.906


<h1>New results for AdaBoost on Decision Tree</h1>

In [25]:
results = {'Depth': [1,3,6,9,12,15,18],
        'Accuracy_Gini': [0.9194,0.9639,0.9654,0.9739, 0.9757,0.9760,0.9734],
        'Accuracy_Entropy': [0.9130,0.9581,0.9657,0.9736,0.9760,0.9762, 0.9736 ]
          }
df = pd.DataFrame(results, columns = ['Depth', 'Accuracy_Gini', 'Accuracy_Entropy'])
df

,Depth,Accuracy_Gini,Accuracy_Entropy
0,1,0.9194,0.9130
1,3,0.9639,0.9581
2,6,0.9654,0.9657
3,9,0.9739,0.9736
4,12,0.9757,0.9760
5,15,0.9760,0.9762
6,18,0.9734,0.9736


<h1>Key Observations:</h1>
<h3>1. The accuracy of AdaBoost on Decision Tree is significantly higher than with just the Decision Tree model. This is to be expected because AdaBoost begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases. This improves accuracy.</h3>
<h3>2. The accuracies of Gini are more than Entropy till depth 9 but then Entropy starts to take over in terms of accuracy values. This is similar to the previous behaviour in accuracies where till depth 6 the accuracies of Gini remained higher than Entropy. This shows that AdaBoost does not change the behaviour of the classifier and just enhances the accuracy.</h3>
<h3>3. Both the trees start to overfit at depth 18 as the accuracy starts decreasing.</h3>